# Adidas: Verkaufsprognose für Badeschlappen

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_09/adidas_simple.ipynb)

**Lernziel:** Grundlagen des Machine Learning - Lineare Regression für saisonale Verkaufsprognosen

**Szenario:** Vorhersage der Badeschlappen-Verkäufe basierend auf Temperatur für verschiedene Sommer-Szenarien

**Lehrbeispiel:** Die Daten sind simuliert und basieren nicht auf realen Zusammenhängen. Es gibt keinen Bezug zu Adidas. :-)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Historische Daten der letzten 3 Jahre (156 Wochen)
np.random.seed(42)
n_wochen = 156  # 3 Jahre

# Saisonale Temperatur (Deutschland)
wochen = np.arange(1, n_wochen + 1)
# Grundtemperatur mit saisonalem Muster
temperatur = 12 + 15 * np.sin(2 * np.pi * wochen / 52) + np.random.normal(0, 3, n_wochen)

# Badeschlappen-Verkäufe: stark temperaturabhängig
# Basis-Verkäufe sehr niedrig, aber steigt exponentiell mit Temperatur
basis_verkaeufe = 50  # Grundverkauf auch im Winter
# Starker Temperatur-Effekt für Badeschlappen
temp_schwelle = 15  # Ab 15°C wird's interessant
# Sicherstellen, dass nur positive Werte für Potenz verwendet werden
temp_diff = np.maximum(temperatur - temp_schwelle, 0)  # Negative Werte auf 0 setzen
temp_effekt = temp_diff ** 1.5 * 40  # Exponentieller Anstieg nur für positive Differenzen

rauschen = np.random.normal(0, 30, n_wochen)
verkaeufe = basis_verkaeufe + temp_effekt + rauschen
verkaeufe = np.maximum(verkaeufe, 10)  # Mindestens 10 Paar pro Woche

df = pd.DataFrame({
    'woche': wochen,
    'temperatur': temperatur.round(1),
    'verkaeufe': verkaeufe.round(0).astype(int)
})

print(f"Historische Daten: {n_wochen} Wochen (3 Jahre)")
print(f"Temperatur-Range: {df['temperatur'].min():.1f}°C bis {df['temperatur'].max():.1f}°C")
print(f"Verkaufs-Range: {df['verkaeufe'].min()} bis {df['verkaeufe'].max()} Paar Badeschlappen/Woche")
print("\nErste 10 Wochen:")
df.head(10)

In [ ]:
# Daten visualisieren
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Zeitreihe der letzten 52 Wochen (1 Jahr)
letztes_jahr = df.tail(52)
ax1.plot(letztes_jahr['woche'], letztes_jahr['temperatur'], 'r-', label='Temperatur', linewidth=2)
ax1_twin = ax1.twinx()
ax1_twin.plot(letztes_jahr['woche'], letztes_jahr['verkaeufe'], 'b-', label='Badeschlappen-Verkäufe', linewidth=2)
ax1.set_xlabel('Woche')
ax1.set_ylabel('Temperatur (°C)', color='red')
ax1_twin.set_ylabel('Verkäufe (Paar)', color='blue')
ax1.set_title('Saisonaler Verlauf: Temperatur und Badeschlappen-Verkäufe (letztes Jahr)')

# Streudiagramm: Temperatur vs. Verkäufe
ax2.scatter(df['temperatur'], df['verkaeufe'], alpha=0.6, color='orange', s=20)
ax2.set_xlabel('Temperatur (°C)')
ax2.set_ylabel('Verkäufe (Paar)')
ax2.set_title('Zusammenhang: Temperatur vs. Badeschlappen-Verkäufe (3 Jahre)')
ax2.axvline(x=15, color='red', linestyle='--', alpha=0.7, label='15°C Schwelle')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Korrelation Temperatur-Verkäufe: {df['temperatur'].corr(df['verkaeufe']):.3f}")
print("🩴 Erkenntnis: Badeschlappen werden hauptsächlich bei warmem Wetter gekauft!")

In [ ]:
# Machine Learning Modell für Badeschlappen-Verkäufe
X = df[['temperatur']]  # Eingangsvariable: Temperatur
y = df['verkaeufe']     # Zielvariable: Verkäufe

# Modell trainieren
model = LinearRegression()
model.fit(X, y)

# Vorhersagen für historische Daten
y_pred = model.predict(X)

# Modell-Performance
r2 = r2_score(y, y_pred)

print("=== BADESCHLAPPEN-VERKAUFSPROGNOSE-MODELL ===")
print(f"Temperatur-Effekt: {model.coef_[0]:.1f} Paar pro °C")
print(f"Basis-Verkäufe: {model.intercept_:.0f} Paar")
print(f"Modell-Güte (R²): {r2:.3f}")

# Beispiel-Vorhersagen mit korrekten DataFrames
print(f"\n🌡️ Vorhersage-Beispiele:")
temp_winter = pd.DataFrame({'temperatur': [10]})
temp_fruehling = pd.DataFrame({'temperatur': [20]})
temp_hochsommer = pd.DataFrame({'temperatur': [30]})

print(f"10°C (Winter): {model.predict(temp_winter)[0]:.0f} Paar")
print(f"20°C (Frühling): {model.predict(temp_fruehling)[0]:.0f} Paar") 
print(f"30°C (Hochsommer): {model.predict(temp_hochsommer)[0]:.0f} Paar")

In [ ]:
# Szenario-Planung für kommenden Sommer
print("=== SZENARIO-PLANUNG FÜR KOMMENDEN SOMMER ===")
print("Basierend auf Wetterprognose-Szenarien:\n")

# Szenario 1: Heißer Sommer
heisser_sommer = [25, 28, 32, 30, 27, 29, 31, 28, 26, 24]  # 10 Wochen Sommer
print("🔥 SZENARIO: Heißer Sommer (10 Wochen)")
heisse_prognosen = []
for woche, temp in enumerate(heisser_sommer, 1):
    temp_df = pd.DataFrame({'temperatur': [temp]})
    prognose = model.predict(temp_df)[0]
    heisse_prognosen.append(prognose)
    print(f"   Woche {woche}: {temp}°C → {prognose:.0f} Paar")

heisser_total = sum(heisse_prognosen)
print(f"   Gesamt (heißer Sommer): {heisser_total:.0f} Paar\n")

# Szenario 2: Kühler Sommer  
kuehler_sommer = [18, 21, 23, 19, 22, 20, 24, 21, 19, 17]  # 10 Wochen
print("❄️ SZENARIO: Kühler Sommer (10 Wochen)")
kuehle_prognosen = []
for woche, temp in enumerate(kuehler_sommer, 1):
    temp_df = pd.DataFrame({'temperatur': [temp]})
    prognose = model.predict(temp_df)[0]
    kuehle_prognosen.append(prognose)
    print(f"   Woche {woche}: {temp}°C → {prognose:.0f} Paar")

kuehler_total = sum(kuehle_prognosen)
print(f"   Gesamt (kühler Sommer): {kuehler_total:.0f} Paar")

# Unterschied berechnen
unterschied = heisser_total - kuehler_total
print(f"\n📊 GESCHÄFTSIMPACT:")
print(f"Unterschied zwischen den Szenarien: {unterschied:.0f} Paar ({(unterschied/kuehler_total*100):.0f}% mehr)")

# Visualisierung der Szenarien
plt.figure(figsize=(12, 8))

# Historische Daten
plt.scatter(df['temperatur'], df['verkaeufe'], alpha=0.3, color='lightgray', label='Historische Daten')
plt.plot(df['temperatur'], y_pred, 'b-', linewidth=2, label='Prognose-Modell')

# Szenarien
plt.scatter(heisser_sommer, heisse_prognosen, color='red', s=100, label='Heißer Sommer', marker='o')
plt.scatter(kuehler_sommer, kuehle_prognosen, color='blue', s=100, label='Kühler Sommer', marker='s')

plt.xlabel('Temperatur (°C)')
plt.ylabel('Verkäufe (Paar)')
plt.title('Badeschlappen-Verkaufsprognose: Sommer-Szenarien')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Business Impact & Strategische Planung

**Geschäftsnutzen der Szenario-Planung:**
- 🩴 **Bestandsplanung:** Bei heißem Sommer-Szenario deutlich mehr Badeschlappen vorbestellen
- ? **Lagerkapazität:** Flexibel auf verschiedene Wetterlagen reagieren
- 💰 **Umsatzoptimierung:** Keine verpassten Verkäufe bei Hitzewellen
- 🏪 **Filialverteilung:** Mehr Bestände in sonnigen Regionen

**Strategische Erkenntnisse:**
- Badeschlappen sind stark wetterabhängig (exponentieller Anstieg ab 15°C)
- Unterschied zwischen heißem und kühlem Sommer kann >50% Umsatz bedeuten
- Langfristige Wetterprognosen ermöglichen bessere Planung

**Nächste Schritte:** 
- Regionale Wetterprognosen einbeziehen
- Marketingkampagnen an Wetter koppeln
- Lagerbestände flexibel gestalten

In [ ]:
# Vergleich: Polynomiale Regression (Grad 2)
from sklearn.preprocessing import PolynomialFeatures

print("=== VERGLEICH: POLYNOMIALE REGRESSION ===")

# Polynomiale Features erstellen (Grad 2)
poly_features = PolynomialFeatures(degree=2)
X_poly = poly_features.fit_transform(X)

# Polynomiales Modell trainieren
poly_model = LinearRegression()
poly_model.fit(X_poly, y)

# Vorhersagen mit polynomialem Modell
y_pred_poly = poly_model.predict(X_poly)

# Performance vergleichen
r2_linear = r2_score(y, y_pred)
r2_poly = r2_score(y, y_pred_poly)

print(f"Lineares Modell R²: {r2_linear:.3f}")
print(f"Polynomiales Modell R²: {r2_poly:.3f}")
print(f"Verbesserung: {((r2_poly - r2_linear) / r2_linear * 100):.1f}%")

# Szenario-Prognosen mit polynomialem Modell
print(f"\n🔥 POLYNOMIALE PROGNOSEN - Heißer Sommer:")
heisse_prognosen_poly = []
for woche, temp in enumerate(heisser_sommer, 1):
    temp_df = pd.DataFrame({'temperatur': [temp]})
    temp_poly = poly_features.transform(temp_df)
    prognose = poly_model.predict(temp_poly)[0]
    heisse_prognosen_poly.append(prognose)
    print(f"   Woche {woche}: {temp}°C → {prognose:.0f} Paar")

heisser_total_poly = sum(heisse_prognosen_poly)
print(f"   Gesamt (polynomial): {heisser_total_poly:.0f} Paar")

print(f"\n❄️ POLYNOMIALE PROGNOSEN - Kühler Sommer:")
kuehle_prognosen_poly = []
for woche, temp in enumerate(kuehler_sommer, 1):
    temp_df = pd.DataFrame({'temperatur': [temp]})
    temp_poly = poly_features.transform(temp_df)
    prognose = poly_model.predict(temp_poly)[0]
    kuehle_prognosen_poly.append(prognose)
    print(f"   Woche {woche}: {temp}°C → {prognose:.0f} Paar")

kuehler_total_poly = sum(kuehle_prognosen_poly)
print(f"   Gesamt (polynomial): {kuehler_total_poly:.0f} Paar")

# Vergleich der Modelle
print(f"\n📊 MODELL-VERGLEICH:")
print(f"Linear - Heißer Sommer: {heisser_total:.0f} Paar")
print(f"Polynomial - Heißer Sommer: {heisser_total_poly:.0f} Paar")
print(f"Unterschied: {heisser_total_poly - heisser_total:.0f} Paar")

# Visualisierung beider Modelle
plt.figure(figsize=(14, 8))

# Sortierte Temperaturen für glatte Kurven
temp_range = np.linspace(df['temperatur'].min(), df['temperatur'].max(), 100)
temp_range_df = pd.DataFrame({'temperatur': temp_range})
temp_range_poly = poly_features.transform(temp_range_df)

# Vorhersagen für Visualisierung
linear_pred_range = model.predict(temp_range_df)
poly_pred_range = poly_model.predict(temp_range_poly)

# Plot
plt.scatter(df['temperatur'], df['verkaeufe'], alpha=0.3, color='lightgray', s=20, label='Historische Daten')
plt.plot(temp_range, linear_pred_range, 'b-', linewidth=2, label=f'Linear (R²={r2_linear:.3f})')
plt.plot(temp_range, poly_pred_range, 'g-', linewidth=2, label=f'Polynomial (R²={r2_poly:.3f})')

# Szenarien
plt.scatter(heisser_sommer, heisse_prognosen, color='red', s=80, label='Linear - Heißer Sommer', marker='o', alpha=0.7)
plt.scatter(heisser_sommer, heisse_prognosen_poly, color='darkred', s=80, label='Polynomial - Heißer Sommer', marker='s')
plt.scatter(kuehler_sommer, kuehle_prognosen, color='blue', s=80, label='Linear - Kühler Sommer', marker='o', alpha=0.7)
plt.scatter(kuehler_sommer, kuehle_prognosen_poly, color='darkblue', s=80, label='Polynomial - Kühler Sommer', marker='s')

plt.xlabel('Temperatur (°C)')
plt.ylabel('Verkäufe (Paar)')
plt.title('Vergleich: Lineare vs. Polynomiale Regression')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Modell-Vergleich & Erkenntnisse

**Warum ist das polynomiale Modell besser?**
- 🔄 **Erfasst Nicht-Linearität:** Exponentieller Anstieg bei höheren Temperaturen
- 📈 **Höhere Genauigkeit:** Besseres R² durch realistische Kurvenform
- 🎯 **Realitätsnähere Prognosen:** Berücksichtigt, dass Badeschlappen-Verkäufe überproportional steigen

**Lerneffekt:**
- Einfache lineare Modelle sind ein guter Start
- Bei erkennbar nicht-linearen Zusammenhängen können polynomiale Features helfen
- Höhere R²-Werte bedeuten zuverlässigere Geschäftsprognosen

**Vorsicht vor Overfitting:** Bei zu hohen Polynomgraden kann das Modell die Trainingsdaten "auswendig lernen" aber schlecht auf neue Daten generalisieren.